In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import sqlite3 as sql
import os
import re
import datetime as datetime

In [2]:
# Look for all file directories in the folder
directory = "./Jan_2010_Oct_2021/"
all_file_lst = []
for root, subdirectories, files in os.walk(directory):
    for file in files:
        all_file_lst.append(os.path.join(root, file))
# Only use csv file with "-street" in the name
needed_file_lst = [i for i in all_file_lst if "-stop-and-search" in i]
needed_file_lst

['./Jan_2010_Oct_2021/2014-04\\2014-04-hampshire-stop-and-search.csv',
 './Jan_2010_Oct_2021/2014-05\\2014-05-hampshire-stop-and-search.csv',
 './Jan_2010_Oct_2021/2014-06\\2014-06-hampshire-stop-and-search.csv',
 './Jan_2010_Oct_2021/2014-07\\2014-07-hampshire-stop-and-search.csv',
 './Jan_2010_Oct_2021/2014-08\\2014-08-hampshire-stop-and-search.csv',
 './Jan_2010_Oct_2021/2014-09\\2014-09-hampshire-stop-and-search.csv',
 './Jan_2010_Oct_2021/2014-10\\2014-10-gwent-stop-and-search.csv',
 './Jan_2010_Oct_2021/2014-10\\2014-10-hampshire-stop-and-search.csv',
 './Jan_2010_Oct_2021/2014-11\\2014-11-gwent-stop-and-search.csv',
 './Jan_2010_Oct_2021/2014-11\\2014-11-hampshire-stop-and-search.csv',
 './Jan_2010_Oct_2021/2014-12\\2014-12-avon-and-somerset-stop-and-search.csv',
 './Jan_2010_Oct_2021/2014-12\\2014-12-cleveland-stop-and-search.csv',
 './Jan_2010_Oct_2021/2014-12\\2014-12-dorset-stop-and-search.csv',
 './Jan_2010_Oct_2021/2014-12\\2014-12-gwent-stop-and-search.csv',
 './Jan_2010_

In [3]:
# Look for names of the sections per file using regular expression
section_lst = []
for i in needed_file_lst:
    section_name = re.search('20[0-9][0-9]-[0-9][0-9]-(.+?)-stop-and-search.csv', i)
    if section_name:
        found = section_name.group(1)
        section_lst.append(found)
section_lst

['hampshire',
 'hampshire',
 'hampshire',
 'hampshire',
 'hampshire',
 'hampshire',
 'gwent',
 'hampshire',
 'gwent',
 'hampshire',
 'avon-and-somerset',
 'cleveland',
 'dorset',
 'gwent',
 'hampshire',
 'humberside',
 'leicestershire',
 'north-wales',
 'north-yorkshire',
 'nottinghamshire',
 'surrey',
 'warwickshire',
 'west-mercia',
 'west-midlands',
 'west-yorkshire',
 'avon-and-somerset',
 'cleveland',
 'dorset',
 'dyfed-powys',
 'greater-manchester',
 'gwent',
 'hampshire',
 'humberside',
 'leicestershire',
 'lincolnshire',
 'norfolk',
 'north-wales',
 'north-yorkshire',
 'nottinghamshire',
 'staffordshire',
 'suffolk',
 'surrey',
 'warwickshire',
 'west-mercia',
 'west-midlands',
 'west-yorkshire',
 'wiltshire',
 'avon-and-somerset',
 'cleveland',
 'dorset',
 'dyfed-powys',
 'greater-manchester',
 'gwent',
 'hampshire',
 'humberside',
 'leicestershire',
 'lincolnshire',
 'norfolk',
 'north-wales',
 'north-yorkshire',
 'northamptonshire',
 'nottinghamshire',
 'staffordshire',
 'su

In [4]:
# Can be used in the future when we want to look for certain section
unique_section_lst = sorted(list(set(section_lst)))
unique_section_lst

['avon-and-somerset',
 'bedfordshire',
 'btp',
 'cambridgeshire',
 'cheshire',
 'city-of-london',
 'cleveland',
 'cumbria',
 'derbyshire',
 'devon-and-cornwall',
 'dorset',
 'durham',
 'dyfed-powys',
 'essex',
 'gloucestershire',
 'greater-manchester',
 'gwent',
 'hampshire',
 'hertfordshire',
 'humberside',
 'kent',
 'lancashire',
 'leicestershire',
 'lincolnshire',
 'merseyside',
 'metropolitan',
 'norfolk',
 'north-wales',
 'north-yorkshire',
 'northamptonshire',
 'northumbria',
 'nottinghamshire',
 'south-wales',
 'south-yorkshire',
 'staffordshire',
 'suffolk',
 'surrey',
 'sussex',
 'thames-valley',
 'warwickshire',
 'west-mercia',
 'west-midlands',
 'west-yorkshire',
 'wiltshire']

In [5]:
# # Create a list consisting of all the file names using all combinations of year, month and section

# # First all sub lists are defined

# # BTP is temporarily ignored
# all_section = ['Avon and Somerset', 'Bedfordshire', 'Cambridgeshire', 'Cheshire', 'City of London', 
#                'Cleveland', 'Cumbria', 'Derbyshire', 'Devon and Cornwall', 'Dorset', 'Durham', 'Dyfed-Powys', 
#                'Essex', 'Gloucestershire', 'Greater Manchester', 'Gwent', 'Hampshire', 'Hertfordshire', 
#                'Humberside', 'Kent', 'Lancashire', 'Leicestershire', 'Lincolnshire', 'Merseyside', 'Metropolitan', 
#                'Norfolk', 'North Wales', 'North Yorkshire', 'Northamptonshire', 'Northumbria', 'Nottinghamshire', 
#                'South Wales', 'South Yorkshire', 'Staffordshire', 'Suffolk', 'Surrey', 'Sussex', 'Thames Valley', 
#                'Warwickshire', 'West Mercia', 'West Midlands', 'West Yorkshire', 'Wiltshire']
# year_lst = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
# month_lst = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

# # Concat year and month to form a list consisting of all possible months
# time_lst = []
# for i in year_lst:
#     for j in month_lst:
#         time_lst.append(i + '-' + j)
# del time_lst[-2: ]
# time_lst = time_lst[11: ]

# # Convert the format of all_section to suit file name
# street_lst = [i.lower().replace(' ', '-') for i in all_section]

# # Convert the above 2 lists to the file list
# file_lst = []
# for i in time_lst:
#     for j in street_lst:
#         file_lst.append(i + "/" + i + "-" + j)
# file_lst

In [6]:
# Create a new sql database file
conn = sql.connect('crime.db')
# It will loop over every file inside Jan_2010_Oct_2021 folder
for i in tqdm(range(len(needed_file_lst))):
    df = pd.read_csv(needed_file_lst[i], parse_dates = ['Date'])
    # Remove all data without geographical data
    df = df[~df['Longitude'].isna()]
    # Create a column with the name of section (For easy searching when using SQL in the future)
    df['Section'] = section_lst[i]
    # Create year, month and day columns (For easy searching and aggregation when using SQL in the future)
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['Year'] = df['Date'].dt.year
    # Convert the datatype of some attributes
    df = df.astype({"Year": int, "Month": int, "Day": int, "Longitude": "float64", "Latitude": "float64"})
    # Rename some columns
    df = df.rename(columns = {'Part of a policing operation': 'Part_of_a_policing_operation', 
                              'Policing operation': 'Policing_operation', 'Age range': 'Age_range', 
                              'Self-defined ethnicity': 'Self_defined_ethnicity', 
                              'Officer-defined ethnicity': 'Officer_defined_ethnicity', 'Object of search': 'Object_of_search', 
                              'Outcome linked to object of search': 'Outcome_linked_to_object_of_search', 
                              'Removal of more than just outer clothing': 'Removal_of_more_than_just_outer_clothing'})
    # Append the dataframe into the SQL database
    if i == 0:
        df.to_sql('outcome_stop', conn, if_exists = 'replace', index = False)
    else:
        df.to_sql('outcome_stop', conn, if_exists = 'append', index = False)

100%|██████████████████████████████████████████████████████████████████████████████| 3438/3438 [02:16<00:00, 25.14it/s]
